In [4]:
import os
import hashlib
from time import gmtime, strftime
from prettytable import PrettyTable
from csv import DictReader, writer
from enum import Enum


'''
Writes data to a csv file 
Parameters:
file_name (str): The path to the file
mode (str): 'w' for writing and 'a' for appending the data
data (list): list of values to be written
delimiter (str): delimiter between the values
'''


def write(file_name, mode, data, delimiter):
    with open(file_name, mode) as file:
        file_writer = writer(file, delimiter=delimiter)
        file_writer.writerow(data)


'''
Reads static values of task i.e.['Task Id', 'Task Owner', 'Task Description', 'Created At']
and filters the data on filter_header
Parameters:
file_name (str): The path to the file
delimiter (str): delimiter between the values
static_field_headers (list): Headers for static fields of Task. 
filter_header (str): Header of the field to filter
filter_value (str): value of the field to filter

Returns:
list: The filtered list of task for a user
'''


def get_static_values(file_name, delimiter, static_field_headers, filter_header, filter_value):
    tasks = []
    with open(file_name, 'r', newline='') as file:
        file_reader = DictReader(file, delimiter=delimiter)
        for row in file_reader:
            if filter_header is not None and filter_value is not None and row[static_field_headers[1]] == filter_value:
                task = Task(row[static_field_headers[0]],
                            user_name=row[static_field_headers[1]],
                            desc=row[static_field_headers[2]],
                            state=TaskState.PENDING.name,
                            created_at=row[static_field_headers[3]],
                            # updated_at is same as created_at when object is created
                            updated_at=row[static_field_headers[3]])
                tasks.append(task)
    return tasks


'''
Reads dynamic values of task i.e.['Task Id', 'Task State', 'Updated At']
Parameters:
file_name (str): The path to the file
delimiter (str): delimiter between the values
variable_field_headers (list): Headers for static fields of Task. 

Returns:
dict: dictionary of {id: task}
'''


def get_variable_values(file_name, delimiter, variable_field_headers):
    task_updates = {}
    with open(file_name, 'r', newline='') as file:
        file_reader = DictReader(file, delimiter=delimiter)
        for row in file_reader:
            task = Task(row[variable_field_headers[0]],
                        state=row[variable_field_headers[1]],
                        updated_at=row[variable_field_headers[2]])
            task_updates[row[variable_field_headers[0]]] = task
    return task_updates


'''
Prints the tasks along with the headers
Parameters:
tasks (list): The list of tasks
headers (list): list of headers
'''


def pretty_print(tasks, headers):

    # Create a PrettyTable object
    table = PrettyTable()
    table.field_names = headers

    for task in tasks:
        table.add_row([task.id, task.desc, task.state, task.created_at, task.updated_at])

    print(table)


'''
Reads last line of the file
Parameters:
file_name (str): The path to the file

Returns:
str: last line of the file
'''


def read_last_line(file_name):
    with open(file_name, 'rb') as file:
        file.seek(-1, 2)  # Jump to the last byte.
        if file.read(1) == b'\n':  # If the file ends with a newline, step back one more byte.
            file.seek(-2, 1)
        while file.read(1) != b'\n':  # Until EOL is found.
            if file.tell() == 1:  # If we're at the start of the file.
                file.seek(0)
                break
            file.seek(-2, 1)  # Step back one byte.
        last_line = file.readline().decode()
        file.close()
        return last_line


'''
Checks if a value is unique in a file for a header
Parameters:
filename (str): The path to the file
delimiter (str): delimiter between the values
value_to_find (str): value to find in a header
value_header (str): header in which value is to be found
 
Returns:
bool: True if value is unique  else False
'''


def is_value_unique(filename, delimiter, value_to_find, value_header):
    with open(filename, 'r') as file:
        reader = DictReader(file, delimiter=delimiter)
        for row in reader:
            if row[value_header] == value_to_find:
                return False
    return True


'''
Matches values ina given file
Parameters:
filename (str): The path to the file
delimiter (str): delimiter between the values
values_to_match (list): values to match in a corresponding header
value_headers (list): corresponding headers for which value is to be matched
 
Returns:
bool: True if value is matched else False
'''


def match_values(filename, delimiter, values_to_match, value_headers):
    value_match = [False, False]
    with open(filename, 'r') as file:
        reader = DictReader(file, delimiter=delimiter)
        for row in reader:
            value_match = [False, False]
            if row[value_headers[0]] == values_to_match[0]:
                value_match[0] = True
                if row[value_headers[1]] == values_to_match[1]:
                    value_match[1] = True
                break
        return value_match


'''
Gets current GMT time in 'YYYY-MM-DD HH:MM:SS GMT' format

Returns:
str: current GMT time in 'YYYY-MM-DD HH:MM:SS GMT' format
'''


def get_curr_time():
    return strftime("%Y-%m-%d %H:%M:%S GMT", gmtime())


'''
Checks if a file is present in file system

Returns:
bool: True if a file is present in file system else False
'''


def is_file_present(file_path):
    return os.path.exists(file_path)


'''
Checks if a file is empty

Returns:
bool: True if a file is empty else False
'''


def is_file_empty(file_path):
    return os.stat(file_path).st_size == 0


class TaskState(Enum):
    PENDING = 1
    COMPLETED = 2
    DELETED = 3


class Task:
    def __init__(self, task_id, user_name=None, desc=None, state=None, created_at=None, updated_at=None):
        self.__id = task_id

        if user_name is not None:
            self.__user_name = user_name

        if desc is not None:
            self.__desc = desc

        if state is not None:
            self.__state = state

        if created_at is not None:
            self.__created_at = created_at

        if updated_at is not None:
            self.__updated_at = updated_at

    # Getter and setter for task_id
    @property
    def id(self):
        return self.__id

    @id.setter
    def id(self, value):
        self.__id = value

    # Getter and setter for user_name
    @property
    def user_name(self):
        return self.__user_name

    @user_name.setter
    def user_name(self, value):
        self.__user_name = value

    # Getter and setter for desc
    @property
    def desc(self):
        return self.__desc

    @desc.setter
    def desc(self, value):
        self.__desc = value

    # Getter and setter for state
    @property
    def state(self):
        return self.__state

    @state.setter
    def state(self, value):
        self.__state = value

    # Getter and setter for created_at
    @property
    def created_at(self):
        return self.__created_at

    @created_at.setter
    def created_at(self, value):
        self.__created_at = value

    # Getter and setter for updated_at
    @property
    def updated_at(self):
        return self.__updated_at

    @updated_at.setter
    def updated_at(self, value):
        self.__updated_at = value

    def __str__(self):  # This method overrides string method
        return "{}, {}, {}, {}, {}, {}".format(
            self.__id, self.__user_name, self.__desc, self.__state.name, self.__created_at, self.__updated_at
        )

    @staticmethod
    def field_to_header_mapping():  # This method maps class fields to the display headers in the csv file.
        return [('__id', 'Task Id'), ('__user_name', 'Task Owner'), ('__desc', 'Task Description'),
                ('__state', 'Task State'), ('__created_at', 'Created At'), ('__updated_at', 'Updated At')]


class User:
    def __init__(self, user_name, password):
        self.__user_name = user_name
        self.__password = password

    def __str__(self):  # This method overrides string method
        return "{}, {}".format(self.__user_name, self.__password)

    @staticmethod
    def field_to_header_mapping():  # This method maps class fields to the display headers in the csv file.
        return [('__user_name', 'Username'), ('__password', 'Password')]


class AuthenticationMode(Enum):
    LOGIN = 1
    REGISTER = 2
    NONE = 3


class AuthenticationManager:

    # This file would store the user details like username, password.
    __user_file_path = "./user.csv"
    __user_delimiter = ","

    # ['Username','Password']
    __user_display_field_list = [User.field_to_header_mapping()[0][1], User.field_to_header_mapping()[1][1]]

    def __init__(self):
        try:
            is_user_file_present = is_file_present(self.__user_file_path)
            create_user_file = (not is_user_file_present) or is_file_empty(self.__user_file_path)
            if create_user_file:
                # Create user.csv with headers
                write(self.__user_file_path, 'w', self.__user_display_field_list, self.__user_delimiter)
        except Exception as e:
            print(f"\nAuth manager initialization failed: {e}")

    def run(self):
        auth_mode = AuthenticationManager.get_auth_mode_input()

        if auth_mode == AuthenticationMode.LOGIN.value:
            return self.__login()
        elif auth_mode == AuthenticationMode.REGISTER.value:
            return self.__register()
        return ""

    def __register(self):
        __username = input("Enter a unique username: ")
        if not is_value_unique(self.__user_file_path, self.__user_delimiter,
                                     __username, self.__user_display_field_list[0]):
            print("\nUsername already exists. User registration failed.")
            return
        password = input("Enter a password: ")
        hashed_password = self.__hash_password(password)
        write(self.__user_file_path, 'a', [__username, hashed_password], self.__user_delimiter)
        print("\nUser registration successful.")
        return __username

    def __login(self):
        __username = input("Enter username: ")
        password = input("Enter password: ")
        hashed_password = self.__hash_password(password)
        value_match = self.__match_credentials(__username, hashed_password)

        if not bool(value_match[0]):  # Matching username
            print(f"\nNo user with username '{__username}'. Login failed.")
            return
        elif not bool(value_match[1]):
            print(f"\nPassword '{password} 'did not match. Login failed.")
            return

        print("\nLogin successful!")
        return __username

    @staticmethod
    def __hash_password(password):
        return hashlib.sha256(password.encode('utf-8')).hexdigest()

    def __match_credentials(self, __username, hashed_password):
        return match_values(self.__user_file_path, self.__user_delimiter,
                                  [__username, hashed_password], self.__user_display_field_list)

    @staticmethod
    def get_auth_mode_input():
        print("\n\nPlease select one of the modes below to authenticate:\n"
              "\tEnter 1 to login\n"
              "\tEnter 2 to register\n"
              "\tEnter 3 to exit"
              )
        while True:
            user_input = input("\nYour selection: ")

            if not isinstance(user_input, str):  # Input has to be a valid string
                print("\nThe selection was invalid. Please enter a valid string.")
                continue

            try:
                user_input = int(user_input)  # Input has to be an integer
            except ValueError:
                print("\nThe selection was invalid. Please enter an integer.")
                continue

            for auth_mode in AuthenticationMode:
                if auth_mode.value == user_input:
                    return user_input
            print("\nThe selection was invalid. Please choose a valid option.")


class TaskManagerAction(Enum):
    ADD_TASK = 1
    LIST_TASKS = 2
    COMPLETE_TASK = 3
    DELETE_TASK = 4
    EXIT = 5


class TaskManager:
    # This file would store the static task details like task_id, task_desc etc.
    __task_file_path = "./task.csv"
    # This file would store the variable task details like task_id, task_desc etc.
    __task_update_file_path = "./task_update.csv"
    __task_delimiter = ","
    __task_description_max_length = 255
    __max_task_id = None
    __task_update_cache = {}
    __user_task_id__cache = {}
    __username = ""

    # ['Task id', 'Task Owner', 'Task Description', 'Created At']
    __task_static_field_list = [Task.field_to_header_mapping()[0][1], Task.field_to_header_mapping()[1][1],
                                Task.field_to_header_mapping()[2][1], Task.field_to_header_mapping()[4][1]]

    # ['Task id', 'Task State', 'Updated At']
    __task_variable_field_list = [Task.field_to_header_mapping()[0][1], Task.field_to_header_mapping()[3][1],
                                  Task.field_to_header_mapping()[5][1]]

    # ['Task id', 'Task Description', 'Task State', 'Created At', 'Updated At']
    __task_display_field_list = [Task.field_to_header_mapping()[0][1], Task.field_to_header_mapping()[2][1],
                                 Task.field_to_header_mapping()[3][1], Task.field_to_header_mapping()[4][1],
                                 Task.field_to_header_mapping()[5][1]]

    def __init__(self, __username):
        try:
            self.__username = __username
            if (is_file_present(self.__task_file_path)
                    and not is_file_empty(self.__task_file_path)):
                # Get max_task_id
                last_line = read_last_line(self.__task_file_path)
                self.__max_task_id = last_line.split(self.__task_delimiter)[0]
                try:
                    self.__max_task_id = int(self.__max_task_id)  # self.__max_task_id has to be a number
                except ValueError:
                    self.__max_task_id = 0

                    # task_update.csv will be recreated if task.csv has only headers
                    write(self.__task_update_file_path, 'w',
                                self.__task_variable_field_list, self.__task_delimiter)
            else:
                # Create task.csv if missing
                # task_update.csv will also be recreated if task.csv is missing or empty
                write(self.__task_file_path, 'w', self.__task_static_field_list, self.__task_delimiter)
                write(self.__task_update_file_path, 'w', self.__task_variable_field_list, self.__task_delimiter)
                self.__max_task_id = 0

            self.__load_cache(self.__username)  # Load cache
        except Exception as e:
            print(f"\nTask manager initialization failed: {e}")

    def __load_cache(self, __username):
        # Load task ids which have been updated
        self.__load_task_update_cache()
        self.__load_user_task_id_cache(__username)

    def __load_task_update_cache(self):
        task_updates = get_variable_values(self.__task_update_file_path, self.__task_delimiter,
                                                 self.__task_variable_field_list)
        for task_id, task in task_updates.items():
            if TaskState.COMPLETED.name == task.state:
                self.__task_update_cache[int(task_id)] = True
            elif TaskState.DELETED.name == task.state:
                self.__task_update_cache[int(task_id)] = False

    def __load_user_task_id_cache(self, __username):
        # Update __user_task_id__cache before returning
        user_printable_tasks = self.__printable_tasks(__username)
        self.__user_task_id__cache[__username] = [int(task.id) for task in user_printable_tasks]

    def __add_task(self, user_name):
        try:
            # Task description can not
            #   1. be empty or
            #   2. have the __task_delimiter
            #   3. have more than 255 characters
            print("\nPlease enter the task description:")
            while True:
                task_desc = input().strip()
                if not task_desc:
                    print("\nTask description can not be empty. Please re-enter: ")
                    continue
                if self.__task_delimiter in task_desc:
                    print(f"\nTask description can not have '{self.__task_delimiter} character'. Please re-enter: ")
                    continue
                if len(task_desc) > self.__task_description_max_length:
                    print(f"\nTask description must be less than {self.__task_description_max_length} characters. "
                          f"Please re-enter: ")
                    continue
                break

            # Save task's static details
            write(self.__task_file_path, 'a',
                        [self.__max_task_id + 1, user_name, task_desc, get_curr_time()], self.__task_delimiter)
            if user_name not in self.__user_task_id__cache:
                self.__user_task_id__cache[user_name] = []
            self.__user_task_id__cache[user_name].append(int(self.__max_task_id + 1))
            self.__max_task_id += 1
            print(f"\nTask #{self.__max_task_id} added successfully.")
        except Exception as e:
            print(f"\nAn error occurred: {e}")

    def __list_tasks(self, user_name):
        try:
            # Pretty Print the tasks
            pretty_print(self.__printable_tasks(user_name), self.__task_display_field_list)
        except Exception as e:
            print(f"\nAn error occurred: {e}")

    def __printable_tasks(self, user_name):
        try:
            tasks_to_show = []
            all_tasks = get_static_values(
                self.__task_file_path,
                self.__task_delimiter,
                self.__task_static_field_list,
                self.__task_static_field_list[1],
                user_name
            )
            task_updates = get_variable_values(self.__task_update_file_path, self.__task_delimiter,
                                                     self.__task_variable_field_list)
            if not task_updates:
                tasks_to_show = all_tasks
            else:
                for task in all_tasks:
                    if task_updates.get(task.id) is not None:  # one task can only have one entry in task_updates
                        if TaskState.DELETED.name == task_updates[task.id].state:
                            continue
                        else:
                            task.state = task_updates[task.id].state
                            task.updated_at = task_updates[task.id].updated_at
                    tasks_to_show.append(task)
            return tasks_to_show
        except Exception as e:
            print(f"\nAn error occurred: {e}")
        return []

    def __update_task(self, new_state, __username):
        try:
            #   0. User has any PENDING/COMPLETED tasks
            #   1. Task id can not be empty
            #   2. Task id has to be an integer
            #   3. Task id can not be < 1
            #   4. Task id can not be greater than max task id currently in the system
            #   5. Task id has to be of a task created by the user

            if __username not in self.__user_task_id__cache or not self.__user_task_id__cache[__username]:
                print("\nNo tasks found. Please add a task before choosing to update.")
                return

            __user_printable_task_ids = self.__user_task_id__cache[__username]

            print("\nPlease enter the task id: ")
            while True:
                task_id = input().strip()
                if not task_id:
                    print("\nTask id can not be empty. Please re-enter: ")
                    continue
                try:
                    task_id = int(task_id)  # task_id to be an integer
                except ValueError:
                    print("\nTask id has to be an integer. Please re-enter: ")
                    continue
                if task_id < 1:
                    print("\nTask id can not be less than 1. Please re-enter: ")
                    continue
                if task_id not in __user_printable_task_ids:
                    print(f"\nNo task found with id #{task_id}. Please re-enter: ")
                    continue
                break

            if task_id in self.__task_update_cache:
                if not bool(self.__task_update_cache[int(task_id)]):
                    print(f"\nTask #{task_id} has already been deleted.")
                else:
                    if TaskState.COMPLETED.name == new_state:
                        print(f"\nTask #{task_id} has already been marked completed.")
                    elif TaskState.DELETED.name == new_state:
                        write(self.__task_update_file_path, 'a',
                                    [task_id, new_state, get_curr_time()],
                                    self.__task_delimiter
                                    )
                        self.__task_update_cache[int(task_id)] = False
                        self.__user_task_id__cache[__username].remove(task_id)
                        print(f"\nTask #{task_id} deleted successfully.")
            else:
                write(self.__task_update_file_path, 'a',
                            [task_id, new_state, get_curr_time()],
                            self.__task_delimiter
                            )
                if TaskState.COMPLETED.name == new_state:
                    self.__task_update_cache[int(task_id)] = True
                    print(f"\nTask #{task_id} marked completed.")
                elif TaskState.DELETED.name == new_state:
                    self.__task_update_cache[int(task_id)] = False
                    self.__user_task_id__cache[__username].remove(task_id)
                    print(f"\nTask #{task_id} deleted successfully.")

        except Exception as e:
            print(f"\nAn error occurred: {e}")

    def run(self):
        while True:
            action_input = TaskManager.get_task_manager_action()
            if action_input == TaskManagerAction.ADD_TASK.value:
                self.__add_task(self.__username)
            elif action_input == TaskManagerAction.LIST_TASKS.value:
                self.__list_tasks(self.__username)
            elif action_input == TaskManagerAction.COMPLETE_TASK.value:
                self.__update_task(TaskState.COMPLETED.name, self.__username)
            elif action_input == TaskManagerAction.DELETE_TASK.value:
                self.__update_task(TaskState.DELETED.name, self.__username)
            elif action_input == TaskManagerAction.EXIT.value:
                print("\nUser logged out.")
                break
            else:
                print("\nPlease choose a valid option.")  # Input has to be a valid selection

    @staticmethod
    def get_task_manager_action():
        print("\n\nPlease select any of the options below:\n"
              "\tEnter 1 to add a task\n"
              "\tEnter 2 to view tasks\n"
              "\tEnter 3 to mark a task as completed\n"
              "\tEnter 4 to delete a task\n"
              "\tEnter 5 to logout"
              )
        while True:
            __task_manager_action = input("\nYour selection: ")

            if not isinstance(__task_manager_action, str):  # Input has to be a valid string
                print("\nThe selection was invalid. Please enter a valid string.")
                continue

            try:
                __task_manager_action = int(__task_manager_action)  # Input has to be an integer
            except ValueError:
                print("\nThe selection was invalid. Please enter an integer.")
                continue

            return __task_manager_action


if __name__ == "__main__":

    print("\n\n#######################################")
    print("#                Welcome!             #")
    print("#######################################")
    auth_manager = AuthenticationManager()  # Start the auth manager
    username = auth_manager.run()  # Auth manager will return an authenticated username
    if username:  # No username means auth failed or the user chose to exit
        task_manager = TaskManager(username)
        task_manager.run()  # Start the task manager
    print("\n\n#######################################")
    print("#                GoodBye!             #")
    print("#######################################\n")




#######################################
#                Welcome!             #
#######################################


Please select one of the modes below to authenticate:
	Enter 1 to login
	Enter 2 to register
	Enter 3 to exit



Your selection:  2
Enter a unique username:  anurag
Enter a password:  saxena



User registration successful.


Please select any of the options below:
	Enter 1 to add a task
	Enter 2 to view tasks
	Enter 3 to mark a task as completed
	Enter 4 to delete a task
	Enter 5 to logout



Your selection:  1



Please enter the task description:


...


No tasks found. Please add a task before choosing to update.


Please select any of the options below:
	Enter 1 to add a task
	Enter 2 to view tasks
	Enter 3 to mark a task as completed
	Enter 4 to delete a task
	Enter 5 to logout



Your selection:  4



No tasks found. Please add a task before choosing to update.


Please select any of the options below:
	Enter 1 to add a task
	Enter 2 to view tasks
	Enter 3 to mark a task as completed
	Enter 4 to delete a task
	Enter 5 to logout



Your selection:  5



User logged out.


#######################################
#                GoodBye!             #
#######################################

